# Introduction to OpenAI Conversations API

The OpenAI Conversations API provides a powerful way to create and manage persistent conversation contexts. Unlike traditional stateless API calls, conversations allow you to store message history, manage multi-turn interactions, and maintain context across multiple API requests. This notebook walks through all the essential operations including creating conversations, managing items, and understanding the core data structures.

## Setup

First, let's import the OpenAI library and initialize our client.

In [ ]:
from openai import OpenAI
import os

# Initialize the client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

## 1. Create a Conversation

Creating a conversation establishes a persistent context where you can store messages and maintain state across multiple interactions. Conversations support optional metadata for organization and can be initialized with up to 20 items at creation time.

### Parameters

- **`items`** (array, optional): Initial messages to include in the conversation. Maximum 20 items per request.
- **`metadata`** (object, optional): Key-value pairs for storing additional information. Up to 16 pairs; keys max 64 chars, values max 512 chars.

In [ ]:
# Create a new conversation with metadata and initial message
conversation = client.conversations.create(
    metadata={"user_id": "user_456", "session": "onboarding"},
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "I need help setting up my account."
        }
    ]
)

print(f"Conversation ID: {conversation.id}")
print(f"Created at: {conversation.created_at}")
print(f"Metadata: {conversation.metadata}")

## 2. Retrieve a Conversation

Retrieving a conversation returns its core metadata and properties. This operation is useful for checking conversation status, accessing metadata, or verifying that a conversation exists before performing operations on it.

### Parameters

- **`conversation_id`** (string, required): The unique identifier of the conversation to retrieve.

In [ ]:
# Retrieve the conversation we just created
retrieved_conversation = client.conversations.retrieve(conversation.id)

print(f"Retrieved conversation: {retrieved_conversation.id}")
print(f"Metadata: {retrieved_conversation.metadata}")

## 3. Update a Conversation

Updating a conversation allows you to modify its metadata without affecting the conversation items. This is particularly useful for tracking conversation state, user preferences, or categorization as the conversation evolves.

### Parameters

- **`conversation_id`** (string, required): The ID of the conversation to update.
- **`metadata`** (map, required): New key-value pairs to replace existing metadata. Same constraints as creation (16 pairs max, 64/512 char limits).

In [ ]:
# Update conversation metadata to track progress
updated_conversation = client.conversations.update(
    conversation.id,
    metadata={
        "user_id": "user_456",
        "session": "onboarding",
        "status": "in_progress",
        "priority": "high"
    }
)

print(f"Updated metadata: {updated_conversation.metadata}")

## 4. Delete a Conversation

Deleting a conversation removes the conversation object itself but preserves the individual items within it. This operation is irreversible and returns a confirmation message upon success.

### Parameters

- **`conversation_id`** (string, required): The ID of the conversation to delete.

In [ ]:
# Note: Uncomment to actually delete the conversation
# deleted_response = client.conversations.delete(conversation.id)
# print(f"Deleted: {deleted_response.deleted}")
# print(f"Deleted conversation ID: {deleted_response.id}")

print("Skipping deletion to preserve conversation for following examples")

## 5. List Items in a Conversation

Listing items retrieves all messages and other content within a conversation. This endpoint supports pagination, ordering, and selective inclusion of additional data like web search sources, code outputs, and image URLs for rich content interactions.

### Parameters

- **`conversation_id`** (string, required): The conversation ID to list items from.
- **`after`** (string, optional): Item ID for pagination; returns items after this ID.
- **`limit`** (integer, optional): Number of items to return (1-100, default 20).
- **`order`** (string, optional): Sort order - `"asc"` (ascending) or `"desc"` (descending, default).
- **`include`** (array, optional): Additional data to include (e.g., `web_search_call.action.sources`, `code_interpreter_call.outputs`, `message.output_text.logprobs`).

In [ ]:
# List items in the conversation
items_list = client.conversations.items.list(
    conversation.id,
    limit=5,
    order="asc"
)

print(f"Total items retrieved: {len(items_list.data)}")
print(f"Has more items: {items_list.has_more}")
print(f"First item ID: {items_list.first_id}")
print(f"Last item ID: {items_list.last_id}")

# Display each item
for item in items_list.data:
    print(f"\nItem {item.id}:")
    print(f"  Type: {item.type}")
    print(f"  Role: {item.role}")
    print(f"  Status: {item.status}")

## 6. Create Items in a Conversation

Creating items adds new messages or content to an existing conversation. You can add up to 20 items in a single request, enabling efficient batch operations for multi-turn conversations or importing historical context.

### Parameters

- **`conversation_id`** (string, required): The conversation to add items to.
- **`items`** (array, required): Array of items to add. Maximum 20 items per request.
- **`include`** (array, optional): Additional fields to include in response (same options as list items).

In [ ]:
# Add multiple messages to the conversation
new_items = client.conversations.items.create(
    conversation.id,
    items=[
        {
            "type": "message",
            "role": "assistant",
            "content": [{"type": "input_text", "text": "I'd be happy to help you set up your account. What specifically would you like assistance with?"}]
        },
        {
            "type": "message",
            "role": "user",
            "content": [{"type": "input_text", "text": "I need to configure my payment method."}]
        }
    ]
)

print(f"Added {len(new_items.data)} new items")
for item in new_items.data:
    print(f"\nCreated item {item.id}:")
    print(f"  Role: {item.role}")
    print(f"  Content: {item.content[0]['text'][:50]}...")

## 7. Retrieve a Specific Item

Retrieving an individual item returns detailed information about a single message or content piece within a conversation. This is useful for inspecting specific interactions or accessing rich content like images or tool outputs.

### Parameters

- **`conversation_id`** (string, required): The conversation containing the item.
- **`item_id`** (string, required): The specific item ID to retrieve.
- **`include`** (array, optional): Additional data to include in the response.

In [ ]:
# Retrieve the first item we created
if len(new_items.data) > 0:
    first_item_id = new_items.data[0].id
    
    retrieved_item = client.conversations.items.retrieve(
        conversation.id,
        first_item_id
    )
    
    print(f"Retrieved item: {retrieved_item.id}")
    print(f"Type: {retrieved_item.type}")
    print(f"Role: {retrieved_item.role}")
    print(f"Status: {retrieved_item.status}")
    print(f"Content: {retrieved_item.content}")

## 8. Delete an Item

Deleting an item removes a specific message or content piece from a conversation. This operation returns the updated conversation object, allowing you to verify the deletion was successful.

### Parameters

- **`conversation_id`** (string, required): The conversation containing the item.
- **`item_id`** (string, required): The item ID to delete.

In [ ]:
# Note: Uncomment to actually delete an item
# if len(new_items.data) > 0:
#     item_to_delete = new_items.data[0].id
#     updated_conv = client.conversations.items.delete(
#         conversation.id,
#         item_to_delete
#     )
#     print(f"Item deleted from conversation: {updated_conv.id}")

print("Skipping item deletion to preserve conversation state")

## 9. The Conversation Object

The conversation object is the core data structure representing a persistent conversation context. Understanding its properties is essential for effective conversation management.

### Conversation Object Structure

- **`id`** (string): Unique identifier for the conversation (e.g., `"conv_123"`).
- **`object`** (string): Always `"conversation"` for this object type.
- **`created_at`** (integer): Unix timestamp (seconds since epoch) when the conversation was created.
- **`metadata`** (object): Custom key-value pairs for storing additional information (max 16 pairs).

In [ ]:
# Examine the conversation object structure
print("Conversation Object Structure:")
print(f"  ID: {conversation.id}")
print(f"  Object Type: {conversation.object}")
print(f"  Created At: {conversation.created_at}")
print(f"  Metadata: {conversation.metadata}")

# Convert timestamp to readable format
from datetime import datetime
creation_time = datetime.fromtimestamp(conversation.created_at)
print(f"  Creation Date: {creation_time.strftime('%Y-%m-%d %H:%M:%S')}")

## 10. The Item List Object

The item list object is returned when listing conversation items. It provides pagination support and metadata about the collection, enabling efficient navigation through large conversations.

### Item List Structure

- **`object`** (string): Always `"list"` for list responses.
- **`data`** (array): Array of conversation item objects.
- **`first_id`** (string): ID of the first item in the current page.
- **`last_id`** (string): ID of the last item in the current page.
- **`has_more`** (boolean): Indicates if additional items exist beyond this page.

In [ ]:
# Demonstrate pagination with the item list
items_page = client.conversations.items.list(
    conversation.id,
    limit=2
)

print("Item List Object Structure:")
print(f"  Object Type: {items_page.object}")
print(f"  Number of Items: {len(items_page.data)}")
print(f"  First Item ID: {items_page.first_id}")
print(f"  Last Item ID: {items_page.last_id}")
print(f"  Has More Items: {items_page.has_more}")

# Demonstrate pagination if more items exist
if items_page.has_more:
    next_page = client.conversations.items.list(
        conversation.id,
        limit=2,
        after=items_page.last_id
    )
    print(f"\nNext Page:")
    print(f"  Items: {len(next_page.data)}")
    print(f"  First ID: {next_page.first_id}")

## Summary

This notebook covered the complete lifecycle of managing conversations with the OpenAI Conversations API:

1. **Creating conversations** with initial items and metadata
2. **Retrieving and updating** conversation properties
3. **Managing conversation items** (create, list, retrieve, delete)
4. **Understanding core data structures** (conversation object and item list)

The Conversations API enables stateful interactions, making it ideal for chatbots, multi-turn assistants, and applications requiring persistent context across sessions.